# From poll to indexes

## Setup

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import os 
import sys 
import json 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [2]:
spark = (
    SparkSession
        .builder
        .appName('poll_to_idxs')
        .getOrCreate()
    )

spark

## CSV loading and cleaning

In [3]:
# the csv poll dir
poll_dir = "../data/poll_data.csv"

# load the csv poll as a dataframe
df = (
    spark.read.format("csv")
    # .option("encoding", "UTF-8")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(poll_dir)
)

# list of columns that will not be considered individually scored
skip_cols = [
    "Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni",
    "Informazioni cronologiche",
    "Quanti anni hai?",
    "Genere",
    "Da quante persone è composto il tuo nucleo familiare?",
    "Occupazione",
    "Quanto è grande la tua azienda?",
    "Da che regione provieni?",
    "Provincia di provenienza",
    "In che regione lavori/studi?",
    "Provincia del luogo di lavoro/studio",
    "Invalidità",
    "Tipo di residenza",
    "Numero di persone con cui convivi",
    "Entrate Familiari Mensili Nette",
    "Entrate Personali Mensili Nette ",
]

# load the questions json
with open("../data/questions.json", "r", encoding="utf-8") as questions_file:
    questions = json.load(questions_file)

# create a question_text:question_idx map, useful to alias the columns
questions_idxs = {questions[q]["question_text"]: q for q in questions.keys()}

# alias the columns (long column names cause bugs and are hard to use)
for k, v in questions_idxs.items():
    df = df.withColumnRenamed(k, v)

df.show(5)

+-------------------------+-----------------------------------------------------------------------------+-------------------+---------+-----------------------------------------------------+--------------------+-------------------------------+------------------------+------------------------+----------------------------+------------------------------------+------------------+--------------------+---------------------------------+-------------------------------+--------------------------------+---+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----+--------------------+--------------------+--------------------+----+-------------------+--------------------+-------+--------------------+--------------------+---+---+------------------+--------------+---------------+----+--------------+------------+------------+--------------------+------------+----+--------------+-------------+---------------

## Computing the scores

In [4]:
# "subtract" the list of unscorable cols from the list of scorable cols
scorable_cols = [item for item in df.columns if item not in skip_cols]

# iterate over the scorable columns to create the scores dictionary
scores = {}
for col in scorable_cols:
    # shortcuts for some useful values
    question_type = questions[col]["question_type"]
    question_score = questions[col]["question_score"]
    question_answers = questions[col]["answers"]

    # skip unscored questions
    if len(question_answers) == 0:
        continue

    # collect the Row values for the current column and access its value (index 0)
    answers = [str(d[0]) for d in df.select(col).collect()]

    # handle score computation differently based on question_type
    row_scores = []
    if question_type in ["basic", "multivalue"]:
        for a in answers:
            row_scores.append(question_answers[a]["answer_score"] * question_score)

        scores[col] = row_scores

    elif question_type == "comma_separated":
        # * Done
        scores[col] = question_answers.split(";")
        # Deprecated code below
        # scores[col] = [0 for i in range(52)]

In [5]:
# create the scores dataframe from the scores dictionary
df_scores = spark.createDataFrame(pd.DataFrame(scores))
df_scores.show(5)

+----+----+-----+----+----+-----+----+----+-----+----+-----+-----+----+-----+---+---+-----+-----+-----+---+---+-----+-----+----+-----+-----+----+----+----+-----+-----+-----+-----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+-------+-------+-------+-------+-------+-------+-------+-----+-----+----+
| S_1| S_2|  S_3| S_4| S_5|  S_6| S_7| S_8|  S_9|V_10| V_11| V_12|C_13| C_14|m_1|m_2|  m_3|  m_4|  m_5|m_6|m_7|  m_8|  m_9|m_10| m_11| m_12|m_13|m_14|m_15| m_16| m_17| m_18| m_19|m_20|eh1|eh4|eh5|ew1|ew4|ww1|ww2|ww3|ww4|wh1|wh2|wh3|wh4|wastew1|wastew2|wastew3|wastew4|wasteh1|wasteh2|wasteh3|wasteh4| ER_1| ER_2|ER_3|
+----+----+-----+----+----+-----+----+----+-----+----+-----+-----+----+-----+---+---+-----+-----+-----+---+---+-----+-----+----+-----+-----+----+----+----+-----+-----+-----+-----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+-------+-------+-------+-------+-------+-------+-------+-----+-----+----+
|37.5|37.5|  0.0|49.5|99.0|150.0|99.0|99.0| 50

In [6]:
# the indexes dictionary, which will be used to compute the indexes dataframe
indexes_cols = {}

## Computing the indexes

### Purchases indexes

In [7]:
df_s = df_scores.select([f"S_{d}" for d in range(1, 10)]).collect()

i_S_col = []
for s in df_s:
    i_S_col.append(sum([val for val in s])/1450)

indexes_cols['i_S'] = i_S_col

### Mobility indexes

In [8]:
df_m1 = df_scores.select(["m_8", "m_9"]).collect()

i_m1_col = []
for s in df_m1:
    m_8, m_9 = s[0], s[1]
    i_m1_col.append(m_9/m_8)

indexes_cols['i_M1'] = i_m1_col

In [9]:
df_m2 = df_scores.select(["m_17", "m_18"]).collect()

i_m2_col = []
for s in df_m2:
    m_17, m_18 = s[0], s[1]
    i_m2_col.append(m_18/m_17)

indexes_cols['i_M2'] = i_m2_col

In [10]:
fam_size_col_label = "Da quante persone è composto il tuo nucleo familiare?"

df_m3 = df_scores.select(["m_8"]).collect()
df_fam = df.select([fam_size_col_label]).collect()

i_m3_col = []
for s in df_m3:
    m_8, f = s[0], int(df_fam[0][0])
    i_m3_col.append(m_8 / f)

indexes_cols["i_M3"] = i_m3_col

In [11]:
df_m4 = df_scores.select(["m_11", "m_12", "m_13"]).collect()

i_m4_col = []
for s in df_m4:
    m_11, m_12, m_13 = s[0], s[1], s[2]
    i_m4_col.append(m_12 / m_11 + m_12 + m_13)

indexes_cols["i_M4"] = i_m4_col

In [12]:
df_m5 = df_scores.select(["m_20"]).collect()

i_m5_col = []
for s in df_m5:
    m_20 = s[0]
    i_m5_col.append(m_20)

indexes_cols["i_M5"] = i_m5_col

In [17]:
df_m6 = df_scores.select(["m_19"]).collect()

i_m6_col = []
for s in df_m6:
    m_19 = s[0]
    i_m6_col.append(m_19)

indexes_cols["i_M6"] = i_m6_col

In [14]:
df_m7 = df_scores.select(["m_4", "m_5"]).collect()

i_m7_col = []
for s in df_m7:
    m_4, m_5 = s[0], s[1]
    i_m7_col.append(m_5 / m_4)

indexes_cols["i_M7"] = i_m7_col

### Energy indexes

In [ ]:
# Systems (home)
df_e1 = df_scores.select(["eh2"]).collect()
i_e1_col = []
for s in df_e1:
    i_e1_col.append(s/5)
indexes_cols["i_e1"] = i_e1_col

In [ ]:
# Systems (work)
df_e2= df_scores.select(["ew2"]).collect()
i_e2_col = []
for s in df_e2:
    i_e2_col.append(s/5)
indexes_cols["i_e2"] = i_e2_col

In [ ]:
# Sustainable Source (home)
df_e3 = df_scores.select(["eh3","eh4"]).collect()
i_e3_col = []
for s in df_e3:
    eh3, eh4 = s[0], s[1]
    i_e3_col.append(eh3 *eh4)
indexes_cols["i_e3"] = i_e3_col


In [ ]:
# Sustainable Source (work)
df_e4 = df_scores.select(["ew3","ew4"]).collect()
i_e4_col = []
for s in df_e4:
    ew3, ew4 = s[0], s[1]
    i_e4_col.append(ew3* ew4)
indexes_cols["i_e4"] = i_e4_col

In [ ]:
# Efficiency Ratio
df_e5 = df_scores.select(["eh1", "eh5"]).collect()
i_e5_col = []
for s in df_e5:
    eh1, eh5 = s[0], s[1]
    i_e5_col.append(eh1/eh5)
indexes_cols["i_e5"] = i_e5_col

In [ ]:
# GreenBuilding (home)
df_e6 = df_scores.select(["eh6"]).collect()
i_e6_col = []
for s in df_e6:
    i_e6_col.append(s/6)
indexes_cols["i_e6"] = i_e6_col

In [ ]:
# GreenBuilding (work)
df_e7 = df_scores.select(["ew5"]).collect()
i_e7_col = []
for s in df_e7:
    i_e7_col.append(s/6)
indexes_cols["i_e7"] = i_e7_col

### Water Index

In [ ]:
# Water (home)
df_water1 = df_scores.select(["wh1","wh2","wh3", "wh4"]).collect()
i_e8_col = []
for s in df_water1:
    wh1, wh2, wh3, wh4 = s[0], s[1], s[2],s[3]
    i_e8_col.append((wh1+ wh2+ wh3+ wh4)/4)
indexes_cols["i_e8"] = i_e8_col

In [ ]:
# Water (work)
df_water2 = df_scores.select(["ww1","ww2","ww3", "ww4"]).collect()
i_e9_col = []
for s in df_water2:
    ww1, ww2, ww3, ww4 = s[0], s[1], s[2],s[3]
    i_e9_col.append((ww1+ ww2+ ww3+ ww4)/4)
indexes_cols["i_e9"] = i_e9_col

### Waste Index

In [ ]:
# Waste (home)
df_waste1 = df_scores.select(["wasteh1", "wasteh2", "wasteh3", "wasteh4"]).collect()
i_e10_col = []
for s in df_waste1:
    wasteh1, wasteh2, wasteh3, wasteh4 = s[0], s[1],s[2],s[3]
    i_e10_col.append((wasteh1 + wasteh2 + wasteh3 + wasteh4)/4)
indexes_cols["i_e10"] = i_e10_col

In [ ]:
# Waste (work)
df_waste2 = df_scores.select(["wastew1", "wastew2", "wastew3", "wastew4"]).collect()
i_e11_col = []
for s in df_waste1:
    wastew1, wastew2, wastew3, wastew4 = s[0], s[1],s[2],s[3]
    i_e11_col.append((wastew1 + wastew2 + wastew3 + wastew4)/4)
indexes_cols["i_e11"] = i_e11_col

## Indexes dataframe

In [18]:
# create the indexes dataframe from the indexes dictionary
df_indexes = spark.createDataFrame(pd.DataFrame(indexes_cols))
df_indexes.show()

+-------------------+------------------+------------------+-----+------------------+----+-----+------------------+
|                i_S|              i_M1|              i_M2| i_M3|              i_M4|i_M5| i_M6|              i_M7|
+-------------------+------------------+------------------+-----+------------------+----+-----+------------------+
| 0.4286206896551724|1.3333333333333333|               1.0|18.75|             101.0| 0.0|  0.0|1.3333333333333333|
|0.39517241379310347|               0.6|               0.2|31.25|              11.0|50.0|100.0|               1.0|
|0.36931034482758623|               0.2|0.3333333333333333|31.25|              10.1| 0.0|  0.0|               0.1|
|0.33482758620689657|1.6666666666666667|               1.0|18.75|              10.1|50.0|  0.0|1.3333333333333333|
|0.31724137931034485|               0.4|               1.0|31.25|10.033333333333333| 0.0| 25.0|               0.5|
|0.38517241379310346|               0.2|               0.2|31.25|             10